In [0]:


def get_notebook_info_from_folder(folder_path):
    """
    Fetches the name, path, and content of all notebooks in a specified folder.

    Args:
        folder_path (str): The absolute path to the workspace folder (e.g., '/Users/my_user/my_project').

    Returns:
        dict: A dictionary where keys are notebook names and values are 
              {'path': str, 'content': str}.
    """
    # 1. Get Workspace URL and Token
    # This assumes you are running in a Databricks Notebook.
    try:
        host = os.environ.get("DATABRICKS_HOST")
        if not host:
            host = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiUrl().get()
            
        token = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()
        
    except Exception as e:
        print(f"Error retrieving Databricks context/token. Ensure you are running in an interactive notebook: {e}")
        return {}

    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }
    
    notebook_data = {}
    
    # --- STEP 1: List Notebooks in the Folder ---
    list_url = urljoin(host, "/api/2.0/workspace/list")
    list_params = {"path": folder_path}
    
    try:
        list_response = requests.get(list_url, headers=headers, params=list_params)
        list_response.raise_for_status()
        objects = list_response.json().get("objects", [])
    except Exception as e:
        print(f"Error listing workspace contents at '{folder_path}': {e}")
        return {}

    # --- STEP 2: Export Content for Each Notebook ---
    export_url = urljoin(host, "/api/2.0/workspace/export")
    
    for obj in objects:
        if obj.get("object_type") == "NOTEBOOK":
            notebook_path = obj["path"]
            notebook_name = os.path.basename(notebook_path)

            export_params = {
                "path": notebook_path,
                "format": "SOURCE" # Exports as raw source code/text
            }
            
            try:
                export_response = requests.get(export_url, headers=headers, params=export_params)
                export_response.raise_for_status()
                
                # Content is base64-encoded, so we need to decode it
                content_base64 = export_response.json().get("content", "")
                notebook_content = base64.b64decode(content_base64).decode("utf-8")
                
                notebook_data[notebook_name] = {
                    "path": notebook_path,
                    "content": notebook_content
                }
            except Exception as e:
                print(f"Error exporting notebook '{notebook_path}': {e}")
                # You can choose to skip or store the error here
                
    return notebook_data

In [0]:
def call_open_ai_api(system_prompt, user_prompt):
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            response_format={"type": "json_object"},
            messages=[
                {
                    "role": "system",
                    "content": system_prompt
                },
                {
                    "role": "user",
                    "content": user_prompt
                }
            ],
            max_tokens=10000,
            temperature=0,
            seed=0
        )

        # Extract results
        result = response.choices[0].message.content
        prompt_tokens = response.usage.prompt_tokens
        completion_tokens = response.usage.completion_tokens

        return result, prompt_tokens, completion_tokens

    except Exception as e:
        print(f"OpenAI API error: {e}")
        return None, None, None


In [0]:
def clean_notebook_content(raw_content: str) -> str:
    """Removes Databricks boilerplate, empty magic commands, and blank lines."""
    
    # 1. Remove standard boilerplate headers
    content = re.sub(r'# Databricks notebook source\s*', '', raw_content)
    
    # 2. Remove standard COMMAND separators and the Python cell separator
    content = re.sub(r'# COMMAND ----------\s*', '\n', content)
    
    # 3. Remove all MAGIC commands and their arguments (e.g., %md, %run, %sql)
    content = re.sub(r'# MAGIC\s+%[a-zA-Z]+\s+.*?\s*|\s*# MAGIC\s+%[a-zA-Z]+\s*|\s*# MAGIC\s*', '', content)

    # 4. Remove excessive blank lines/whitespace created by the above removals
    content = re.sub(r'\n{2,}', '\n', content).strip()
    
    return content